In [13]:
import torch
from models import NCF3, GMF

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [14]:
# model = GMF(6040, 3416, 128)
# model = NCF(6040, 3416, 128, [128, 64, 32, 16], 1)
model = NCF3(6040, 3416, 128, 4, 1)
model.load_state_dict(torch.load('saved_models/NCF'))

<All keys matched successfully>

In [15]:
model.eval()

NCF3(
  (embed_user_GMF): Embedding(6040, 128)
  (embed_item_GMF): Embedding(3416, 128)
  (embed_user_MLP): Embedding(6040, 1024)
  (embed_item_MLP): Embedding(3416, 1024)
  (out_act): Sigmoid()
  (MLP_layers): Sequential(
    (0): Dropout(p=1, inplace=False)
    (1): Linear(in_features=2048, out_features=1024, bias=True)
    (2): ReLU()
    (3): Dropout(p=1, inplace=False)
    (4): Linear(in_features=1024, out_features=512, bias=True)
    (5): ReLU()
    (6): Dropout(p=1, inplace=False)
    (7): Linear(in_features=512, out_features=256, bias=True)
    (8): ReLU()
    (9): Dropout(p=1, inplace=False)
    (10): Linear(in_features=256, out_features=128, bias=True)
    (11): ReLU()
  )
  (predict_layer): Linear(in_features=256, out_features=1, bias=True)
)

In [4]:
from data import TargetData

data = TargetData()

In [16]:
data.df

,uid_old,mid_old,rating,date,uid,mid,latest
0,1,1193,1,2000-12-31 22:12:40,0,0,42.0
1,1,661,1,2000-12-31 22:35:09,0,1,23.0
2,1,914,1,2000-12-31 22:32:48,0,2,28.0
3,1,3408,1,2000-12-31 22:04:35,0,3,47.0
4,1,2355,1,2001-01-06 23:38:11,0,4,4.0
...,...,...,...,...,...,...,...
999606,6040,1091,1,2000-04-26 02:35:41,6039,772,161.0
999607,6040,1094,1,2000-04-25 23:21:27,6039,1106,293.0
999608,6040,562,1,2000-04-25 23:19:06,6039,365,305.0
999609,6040,1096,1,2000-04-26 02:20:48,6039,152,234.0


In [17]:
from random import choice

row = data.df.sample(1)
row

,uid_old,mid_old,rating,date,uid,mid,latest
379458,2221,608,1,2000-11-19 02:44:07,2220,51,76.0


In [18]:
user = row.uid_old.item()

In [19]:
all_user_likes = data.df[data.df.uid_old==user].copy(deep=True)
not_liked = data.movies - set(all_user_likes.mid_old.values)
aul = all_user_likes.sort_values(by='mid_old').reset_index(drop=True)
aul.head()


,uid_old,mid_old,rating,date,uid,mid,latest
0,2221,1,1,2000-11-19 02:54:36,2220,40,43.0
1,2221,17,1,2000-11-19 02:50:00,2220,417,66.0
2,2221,24,1,2000-11-19 02:15:11,2220,359,133.0
3,2221,32,1,2000-11-19 02:09:39,2220,376,153.0
4,2221,36,1,2000-11-19 02:48:40,2220,383,71.0


In [20]:
import numpy as np
# [x for _, x in sorted(zip(Y, X))]
# list(data.movies)
y = model(
    torch.LongTensor(np.full(data.num_movies, user)).to(device),
    torch.LongTensor(list(data.movies)).to(device)
)

In [21]:
import pandas as pd
cols = list(zip(list(data.movies),y.cpu().detach().numpy().squeeze()))

scores = pd.DataFrame(cols, columns=['mid_old', 'score'])

In [22]:
df = pd.merge(scores, aul, how='inner', on=['mid_old', 'mid_old'])




In [23]:
df.sort_values(by='score')

,mid_old,score,uid_old,rating,date,uid,mid,latest
107,2289,0.000000e+00,2221,1,2000-11-19 02:51:34,2220,275,55.0
147,3175,1.039223e-25,2221,1,2000-11-19 02:09:39,2220,733,151.0
91,1909,2.020170e-24,2221,1,2000-11-19 02:14:07,2220,251,137.0
138,2908,1.938922e-23,2221,1,2000-11-19 02:53:49,2220,372,47.0
144,3033,7.530455e-22,2221,1,2000-11-19 02:34:39,2220,864,91.0
...,...,...,...,...,...,...,...,...
51,1196,1.000000e+00,2221,1,2000-11-19 02:23:29,2220,124,113.0
52,1200,1.000000e+00,2221,1,2000-11-19 02:23:29,2220,714,115.0
53,1210,1.000000e+00,2221,1,2000-11-19 02:24:48,2220,64,110.0
55,1240,1.000000e+00,2221,1,2000-11-19 02:23:29,2220,217,117.0


In [25]:
df.rating.value_counts()

1    151
Name: rating, dtype: int64

In [26]:
aul.rating.value_counts()

1    162
Name: rating, dtype: int64

In [27]:
data.testing_data.rating.value_counts()

0    604000
1      6040
Name: rating, dtype: int64